In [1]:
import numpy as np 
import pandas as pd 
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform

from keras.layers import Input, Dense, Reshape, Flatten, Dropout, concatenate, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, Subtract
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D ,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, Nadam, RMSprop
import datetime
import sys
import os

import gc
from copy import deepcopy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, TensorBoard
from keras import backend as K
from keras import initializers
from keras import regularizers
from sklearn.model_selection import TimeSeriesSplit
from keras.models import load_model  
print(os.listdir("../input"))

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Using TensorFlow backend.


['england-traffic-volume']
/kaggle/input/england-traffic-volume/trafficV_M.csv


加载  预处理数据

In [2]:
# 交通矩阵为 matrix_length*matrix_length
matrix_length = 32

matrix_df = pd.read_csv('../input/england-traffic-volume/trafficV_M.csv', index_col=0, parse_dates=True)

In [3]:
def createTrainArray(week_history_num=0, minute_history_num=0):
    week_delta_list = [pd.Timedelta(i+1, unit='W') for i in range(week_history_num)]
    minute_delta_list = [pd.Timedelta((i+1)*15, unit='m') for i in range(minute_history_num)]
    # 参考历史数据时间点list
    delta_list = week_delta_list+minute_delta_list
    print(delta_list)
    
    set_up_time = pd.Timedelta(week_history_num, unit='W')
    # 根据历史数据选取多少，重新构建数据集
    # 相当于去除最开始week_history_num个周的数据，因为这些数据无法找到更前的数据
    train_df = matrix_df.truncate(before=matrix_df.index.min() + set_up_time)
    
    train_ago_array_tuple = tuple([np.array(matrix_df.loc[train_df.index - i]).reshape(-1, matrix_length, matrix_length, 1) for i in delta_list])
    train_df = np.array(train_df).reshape(-1, matrix_length, matrix_length, 1)
    # concatenate保持 待修复数据在前，参考历史数据在后。与random_mask函数生成mask相一致
    train_array = np.concatenate((train_df,)+train_ago_array_tuple, axis=3)
    print(train_array.shape)
    return train_array


def normalization(matrix):
    for i in range(len(matrix)):
        for j in range(matrix.shape[-1]):
            cur_time = matrix[i][:, :, j]
#             mean_val = cur_time.mean()
            mx = cur_time.max()
            mn = cur_time.min()
            matrix[i][:, :, j] = np.divide((cur_time-mn), (mx-mn))
    return matrix

In [4]:
week_history_num = 2
minute_history_num = 3

channel_num = week_history_num +minute_history_num +1
smooth_time = channel_num-1

# train_array为(16704, 32, 32, 3)，16704个矩阵，32*32采集点，3从上到下为当前时间，上一周，上一15min
train_array = createTrainArray(week_history_num, minute_history_num)
X_train, X_test = train_test_split(train_array, test_size = 0.2, random_state=42, shuffle=False)
# X_train, X_val = train_test_split(train_array, test_size = 0.1, random_state=42, shuffle=False) # 不shuffle可用于查看数据正确性

[Timedelta('7 days 00:00:00'), Timedelta('14 days 00:00:00'), Timedelta('0 days 00:15:00'), Timedelta('0 days 00:30:00'), Timedelta('0 days 00:45:00')]
(16032, 32, 32, 6)


In [5]:
X_train.shape, X_test.shape

((12825, 32, 32, 6), (3207, 32, 32, 6))

In [6]:
MAX_BATCH_SIZE = 32
epoch_steps = X_train.shape[0] // MAX_BATCH_SIZE
test_steps = X_test.shape[0] // MAX_BATCH_SIZE
epoch_steps, test_steps

(400, 100)

In [7]:
# 加载数据
def load_data(volume_matrix, batch_size=MAX_BATCH_SIZE):
    n_batches=batch_size
    len_of_matrix = len(volume_matrix)

    batch_i = 0
    while ((batch_i+1)*batch_size < len_of_matrix):
        batch_matrix = volume_matrix[batch_i*batch_size: (batch_i+1)*batch_size]
        true_volume, history_volume = batch_matrix[:, :, :, :1], batch_matrix[:, :, :, 1:]
#         history_volume = normalization(history_volume)
        batch_i+=1

        yield true_volume, history_volume

In [8]:
import math

def l2(y_true, y_pred):
    return math.sqrt(np.sum(np.mean(np.square(y_true - y_pred), axis=0))/1024)

模型

In [9]:
def BatchActivate(x, bn=True):
    if bn:
        x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [10]:
# def build_generator():      

#     # INPUTS
#     input_matrix = Input(shape=history_volume_shape)
#     # kernel_init = initializers.he_normal()
#     # bias_init = initializers.he_normal()
#     kernel_init = 'glorot_uniform'
#     bias_init = 'zeros'

#     # kernel_init = initializers.he_uniform()
#     # bias_init = initializers.he_uniform()
#     kernel_regul = regularizers.l2(1)
#     activity_regul = regularizers.l2(1)

#     # ENCODER
#     def encoder_layer(img_in, filters, kernel_size, bn=True, resid=True):
#         # conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), padding='same')(img_in)
#         conv = Conv2D(filters, (kernel_size, kernel_size), padding="same",
#            strides=1,kernel_initializer='glorot_uniform')(img_in)
#         if bn:
#             conv = BatchNormalization()(conv)
# #         conv = Activation('relu')(conv)
#         conv = LeakyReLU(alpha=0.1)(conv)

#         conv = Conv2D(filters, (kernel_size, kernel_size), padding="same",
#            strides=1,kernel_initializer='glorot_uniform')(conv)
#         conv = BatchNormalization()(conv)
# #         conv = Activation('relu')(conv)
#         conv = LeakyReLU(alpha=0.1)(conv)
#         return conv

#     # DECODER
#     def decoder_layer(img_in, filters, kernel_size, bn=True, resid=True):
#         conv = Conv2D(filters, (3, 3), padding="same",
#            strides=1,kernel_initializer='glorot_uniform')(img_in)
#         conv = BatchNormalization()(conv)
#         conv = LeakyReLU(alpha=0.1)(conv)
# #         conv = Activation('relu')(conv)

#         conv = Conv2D(filters//2, (3, 3), padding="same",
#            strides=1,kernel_initializer='glorot_uniform')(conv)
# #             if bn:
#         conv = BatchNormalization()(conv)
# #         conv = Activation('relu')(conv)
#         conv = LeakyReLU(alpha=0.1)(conv)

#         conv = UpSampling2D(size = (2,2))(conv)
#         return conv

#     encoder_layer.counter = 0
#     conv1 = encoder_layer(input_matrix, 32, 3, bn=False)
#     pool1 = AveragePooling2D(pool_size=(2, 2))(conv1)

#     conv2 = encoder_layer(pool1, 64, 3, bn=True)
#     pool2 = AveragePooling2D(pool_size=(2, 2))(conv2)

#     conv3 = encoder_layer(pool2, 128, 3, bn=True)
#     pool3 = AveragePooling2D(pool_size=(2, 2))(conv3)

# #         conv4 = encoder_layer(pool3, 256, 3, bn=True)
# #         pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

#     conv5 = decoder_layer(pool3, 256, 3, bn=True)
#     merge1 = Concatenate()([conv3,conv5])

#     conv6 = decoder_layer(merge1, 128, 3, bn=True)
#     merge2 = Concatenate()([conv2,conv6])

#     conv7 = decoder_layer(merge2, 64, 3, bn=True)
#     merge3 = Concatenate()([conv1,conv7])

# #         conv8 = decoder_layer(merge3, 32, 3, bn=True)
# #         merge4 = Concatenate()([conv1,conv8])

#     conv9 = encoder_layer(merge3, 32, 3, bn=False)
    
#     model_output = Conv2D(1, (1, 1), 
#            use_bias=False, padding="same",activation="linear",
#            strides=1,kernel_initializer='glorot_uniform',
#            name='block9_conv3')(conv9)

#     # Setup the model inputs / outputs
#     model = Model(inputs=input_matrix, outputs=model_output)

#     # Compile the model
#     model.compile(optimizer = Adam(lr=learn_rate), loss='mse')

#     return model

In [11]:
size = (3, 3)
g_filters_base = 32
DropoutRatio = 0.5
learn_rate_g = 0.001
learn_rate_d = 0.001
learn_rate_c = 0.001

# channels = 3
matrix_shape = (matrix_length, matrix_length, channel_num)
true_volume_shape = (matrix_length, matrix_length, 1)
history_volume_shape = (matrix_length, matrix_length, channel_num-1)

kernel_init = 'glorot_uniform'
bias_init = 'zeros'
kernel_regul = regularizers.l2(1)
activity_regul = regularizers.l2(1)

In [12]:
# ResNet block
def identity_block(X, filters, f):

    F1, F2 = filters

    X_shortcut = X

    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F1, kernel_size=(f, f), strides=(1, 1), padding='same',
               kernel_initializer=kernel_init, bias_initializer=bias_init,
              kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(X)

    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               kernel_initializer=kernel_init, bias_initializer=bias_init,
              kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# ENCODER
def encoder_layer(img_in, filters, kernel_size, bn=True, resid=True):
    # conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), padding='same')(img_in)
    conv = img_in
    if bn:
        conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
#             conv = MaxPooling2D((2, 2))(conv)


    if resid:
        conv = identity_block(conv, (filters, filters), kernel_size)

    return conv

# DECODER
def decoder_layer(img_in, e_conv, filters, kernel_size, bn=True, resid=True):
    # up_img = UpSampling2D(size=(2,2))(img_in)
    up_img = img_in
    concat_img = Concatenate(axis=3)([e_conv,up_img])
    conv = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1), padding='same',
                  kernel_initializer=kernel_init, bias_initializer=bias_init,
              kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(concat_img)
    if bn:
        conv = BatchNormalization()(conv)
    conv = LeakyReLU(alpha=0)(conv)

    if resid:
        conv = identity_block(conv, (filters, filters), kernel_size)
    return conv



def build_generator():      

    # INPUTS
    history_traffic_volume = Input(shape=history_volume_shape)

    # kernel_init = initializers.he_normal()
    # bias_init = initializers.he_normal()
    kernel_init = 'glorot_uniform'
    bias_init = 'zeros'

#         kernel_init = initializers.he_uniform()
#         bias_init = 'Orthogonal'
    kernel_regul = regularizers.l2(1)
    activity_regul = regularizers.l2(1)

    filters_base = 32
    e_conv1_head = Conv2D(filters=filters_base, kernel_size=3, strides=1, padding='same',
                          kernel_initializer=kernel_init, bias_initializer=bias_init,
                  kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(history_traffic_volume)
#         e_conv1_head = Conv2D(filters=filters_base*1, kernel_size=3, strides=1, padding='same',
#                               kernel_initializer=kernel_init, bias_initializer=bias_init,
#                       kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(e_conv1_head)
    e_conv1_tail = AveragePooling2D((2, 2))(e_conv1_head)
    e_conv1 = encoder_layer(e_conv1_tail, filters_base, 3, bn=False)

    e_conv2_head = Conv2D(filters=filters_base*2, kernel_size=3, strides=1, padding='same',
                          kernel_initializer=kernel_init, bias_initializer=bias_init,
                  kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(e_conv1)
    e_conv2_tail = AveragePooling2D((2, 2))(e_conv2_head)
    e_conv2 = encoder_layer(e_conv2_tail, filters_base*2, 3)

    e_conv3_head = Conv2D(filters=filters_base*4, kernel_size=3, strides=1, padding='same',
                          kernel_initializer=kernel_init, bias_initializer=bias_init,
                  kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(e_conv2)
    e_conv3_tail = AveragePooling2D((2, 2))(e_conv3_head)
    d_conv3_head = encoder_layer(e_conv3_tail, filters_base*4, 3)
    resid1 = Subtract()([e_conv3_tail, d_conv3_head])
    d_conv3_tail = UpSampling2D(size=(2, 2))(resid1)


    d_conv4_head = decoder_layer(d_conv3_tail, e_conv3_head, filters_base*2, 3)
    resid2 = Subtract()([d_conv4_head, e_conv2_tail])
    d_conv4_tail = UpSampling2D(size=(2, 2))(resid2)


    d_conv5_head = decoder_layer(d_conv4_tail, e_conv2_head, filters_base*1, 3)
    resid3 = Subtract()([d_conv5_head, e_conv1_tail])
    d_conv5_tail = UpSampling2D(size=(2, 2))(resid3)

    d_conv6_head = decoder_layer(d_conv5_tail, e_conv1_head, filters_base//2, 3, bn=False)


    outputs = Conv2D(1, 1, activation = 'relu', kernel_initializer=kernel_init, bias_initializer=bias_init,
                  kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(d_conv6_head)

    # Setup the model inputs / outputs
    model = Model(inputs=history_traffic_volume, outputs=outputs)

    # Compile the model
    model.compile(
        optimizer = Adam(lr=learn_rate_g),
        loss='mse'
    )

    return model

In [13]:
d_filters_base = 32
# Input shape

# Discriminator
def build_discriminator():
    def d_layer(layer_input, filters, f_size=3, bn=True, stride=1):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=stride, padding='same', kernel_initializer=kernel_init, bias_initializer=bias_init,
                  kernel_regularizer=kernel_regul, bias_regularizer=activity_regul)(layer_input)
        if bn:
            d = BatchNormalization()(d)
        d = LeakyReLU(alpha=0.1)(d)
        return d
    
    matrix_A = Input(shape=true_volume_shape)
    matrix_B = Input(shape=history_volume_shape)

    # Concatenate image and conditioning image生成输入对象
    combined_matrix = Concatenate(axis=-1)([matrix_A, matrix_B])

    d1 = d_layer(combined_matrix, d_filters_base, bn=False)
    d2 = d_layer(d1, d_filters_base*2, stride=2)
#     d2 = AveragePooling2D((2, 2))(d2)
    d3 = d_layer(d2, d_filters_base*4, stride=2)
#     d3 = AveragePooling2D((2, 2))(d3)
    d4 = d_layer(d3, d_filters_base*8, stride=2)
#     d4 = AveragePooling2D((2, 2))(d4)
    d4 = d_layer(d4, d_filters_base*4)
    d5 = d_layer(d4, d_filters_base*2)
    d6 = d_layer(d5, d_filters_base*1)
    
    validity = Conv2D(1, kernel_size=3, strides=1, padding='same')(d6)

    model = Model([matrix_A, matrix_B], validity)
    model.compile(loss='mse', optimizer=Adam(lr=learn_rate_d), metrics=['mse'])
    return model

In [14]:
# 计算D输出valid大小（PatchGAN）
patch = 4
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 64
df = 64

optimizer = Adam(lr=learn_rate_c)

# Build and compile the discriminator
discriminator = build_discriminator()

# Build the generator
generator = build_generator()

# Input images and their conditioning images
true_volume = Input(shape=true_volume_shape)
history_volume = Input(shape=history_volume_shape)

# By conditioning on B generate a fake version of A
forecast_volume = generator(history_volume)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminators determines validity of translated images / condition pairs
valid_gan = discriminator([forecast_volume, history_volume])

combined = Model(inputs=[true_volume, history_volume], outputs=[valid_gan, forecast_volume])
combined.compile(loss=['mse', 'mse'], optimizer=optimizer)

In [15]:
lr_step = []
l2_validation = []

def train(train_matrix, epochs, batch_size=MAX_BATCH_SIZE, learn_rate=0.01):

    start_time = datetime.datetime.now()
    print("train start "+str(start_time))

    # Adversarial loss ground truths
#     valid = np.ones((MAX_BATCH_SIZE,) + disc_patch)+np.random.rand(MAX_BATCH_SIZE, patch, patch, 1)/5
#     fake = np.zeros((MAX_BATCH_SIZE,) + disc_patch)+np.random.rand(MAX_BATCH_SIZE, patch, patch, 1)/5
    valid = np.ones((MAX_BATCH_SIZE,) + disc_patch)
    fake = np.zeros((MAX_BATCH_SIZE,) + disc_patch)

    for epoch in range(epochs):
        if epoch>9 and epoch % 2 == 0 and epoch != 0:
            if learn_rate>0.001:
                if epoch%3==0:
                    learn_rate = 0.9*learn_rate
                else:
                    learn_rate = 0.9*learn_rate
                K.set_value(generator.optimizer.lr, learn_rate)
                K.set_value(discriminator.optimizer.lr, learn_rate)

        for batch_i, (true_volume, history_volume) in enumerate(load_data(train_matrix,batch_size)):
            # true_volume 真实待预测路网交通量  history_volume 路网交通量历史数据
            #  训练 Discriminator

            # 根据历史数据生成预测数据
            forecast_volume = generator.predict(history_volume)

            # 训练 the discriminators (original images = real / generated = Fake)
            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch([true_volume, history_volume], valid)
            d_loss_fake = discriminator.train_on_batch([forecast_volume, history_volume], fake)
            discriminator.trainable = False
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  训练 Generator
            g_loss = combined.train_on_batch([true_volume, history_volume], [valid, true_volume])

            elapsed_time = datetime.datetime.now() - start_time

        # Plot the progress
        y_pred = generator.predict(X_test[:, :, :, 1:])
        y_true = X_test[:, :, :, :1]

        l2_epoch_validation = l2(y_true, y_pred)
        lr_step.append(K.get_value(discriminator.optimizer.lr))
        l2_validation.append(l2_epoch_validation)
        if epoch%1==0:
#             print("discriminator lr:"+ str(K.get_value(discriminator.optimizer.lr)))
            print ("[Epoch %d/%d]  [D loss: %f, mse: %f] [G loss: %f] time: %s" % (epoch+1, epochs,
                                                                    d_loss[0], l2_epoch_validation,
                                                                    g_loss[0],
                                                                    elapsed_time))
        # If at show interval => show generated image samples
#             if epoch % show_interval == 0:
#                     show_images(dataset_name,epoch, batch_i)

In [ ]:
train(X_train, epochs=50, batch_size=MAX_BATCH_SIZE, learn_rate=learn_rate_c)

train start 2019-10-18 06:16:15.904114
[Epoch 1/50]  [D loss: 0.324831, mse: 52.668378] [G loss: 2146.386963] time: 0:00:40.477671
[Epoch 2/50]  [D loss: 0.594125, mse: 27.318270] [G loss: 1092.804810] time: 0:01:04.844606
[Epoch 3/50]  [D loss: 0.331617, mse: 26.202221] [G loss: 970.848145] time: 0:01:29.132854
[Epoch 4/50]  [D loss: 0.473915, mse: 29.336246] [G loss: 834.681641] time: 0:01:53.097983
[Epoch 5/50]  [D loss: 0.435174, mse: 27.064365] [G loss: 806.924255] time: 0:02:17.236074
[Epoch 6/50]  [D loss: 0.303389, mse: 24.687080] [G loss: 749.700684] time: 0:02:41.274151
[Epoch 7/50]  [D loss: 0.342476, mse: 24.143380] [G loss: 715.082458] time: 0:03:05.173190
[Epoch 8/50]  [D loss: 0.591660, mse: 23.719183] [G loss: 719.846558] time: 0:03:29.319154
[Epoch 9/50]  [D loss: 0.273295, mse: 23.340588] [G loss: 691.741333] time: 0:03:54.003508
[Epoch 10/50]  [D loss: 0.291517, mse: 23.795860] [G loss: 766.389282] time: 0:04:18.034060
[Epoch 11/50]  [D loss: 0.406804, mse: 22.541907

In [ ]:
# generator.save_weights('/kaggle/working/generatorModel.h5')
# generator.load_weights('/kaggle/working/generatorModel.h5')

In [ ]:
y_pred = generator.predict(X_test[:, :, :, 1:])
y_true = X_test[:, :, :, :1]

l2(y_true, y_pred)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

y = y_true.reshape(-1,)[1600:1700]
x = np.linspace(0, len(y), len(y))

yi = y_pred.reshape(-1,)[1600:1700]
xi = np.linspace(0, len(yi), len(yi))
fig, ax = plt.subplots(figsize=(25, 6))
# ax.plot(x, y, '.', linewidth=1, markersize=10)
lines = plt.plot(xi, yi, 'k^--', x, y, 'ro-',linewidth=1, markersize=6)
plt.show()

In [ ]:
lr_step

In [ ]:
yi = l2_validation
xi = np.linspace(0, len(yi), len(yi))

y = [i*10000 for i in lr_step]
x = np.linspace(0, len(y), len(y))

fig, ax = plt.subplots(figsize=(6, 6))
lines = plt.plot(x, y, 'ko-', xi, yi, 'k^--', linewidth=1, markersize=6)
plt.show()

In [ ]:
y = lr_step
x = np.linspace(0, len(y), len(y))
# fig, ax = plt.subplots(figsize=(6, 6))
# lines = plt.plot(x, y, 'ko-', linewidth=1, markersize=6)
lr_step

In [ ]:
y = y_true.reshape(-1,)[3600:3700]
x = np.linspace(0, len(y), len(y))

yi = y_pred.reshape(-1,)[3600:3700]
xi = np.linspace(0, len(yi), len(yi))
fig, ax = plt.subplots(figsize=(25, 6))
ax.plot(x, y, '.', linewidth=1, markersize=10)
lines = plt.plot(xi, yi, 'k^--', x, y, 'ro-',linewidth=1, markersize=6)
plt.show()

In [ ]:
start_time = 0
y = y_true.reshape(-1,)[start_time: start_time+100]
x = np.linspace(0, len(y), len(y))

yi = y_pred.reshape(-1,)[start_time: start_time+100]
xi = np.linspace(0, len(yi), len(yi))
fig, ax = plt.subplots(figsize=(25, 6))
# ax.plot(x, y, '.', linewidth=1, markersize=10)
lines = plt.plot(xi, yi, 'k^--', x, y, 'ro-',linewidth=1, markersize=6)
plt.show()

In [ ]:
y_pred

In [ ]:
y_true